## prerequisites:

- Beautifulsoup
- selenium
- pandas
- webdriver manager
- openpyxl

In [1]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import urllib3
import re
from bs4 import BeautifulSoup
import csv
http=urllib3.PoolManager()

In [8]:
driver = webdriver.Chrome(ChromeDriverManager().install())

#The website that consists of all the hackathons and their links
driver.get("https://unstop.com/hackathons?filters=,all,all,all&types=teamsize,payment,eligible,oppstatus")

#for just testing purposes the number of scrolls are set to 3, we can increase it any number depending on the 
# hackathons we want to extract contact information for.
ScrollNumber = 3
for i in range(1,ScrollNumber):
    driver.execute_script("window.scrollTo(1,50000)")
    time.sleep(5)

# DS.html has the html code for the infinite scrolling page    
file = open('DS.html', 'w')
file.write(driver.page_source)
file.close()

driver.close()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 96.0.4664
[WDM] - Get LATEST chromedriver version for 96.0.4664 google-chrome
[WDM] - Driver [/home/rohit/.wdm/drivers/chromedriver/linux64/96.0.4664.45/chromedriver] found in cache


## Scrape the HTML code and extract links for all the hackathons and output to excel

In [14]:
def scrape(file_name='DS.html'):
    link_list = []
    main_dict = {}
    data = open(file_name,'r')
    soup = BeautifulSoup(data, 'html.parser')
    for link in soup.find_all('a', {'class': re.compile(r"listing order-\d id-[0-9]+ full_banner ng-star-inserted", re.IGNORECASE)}):
        link_list.append('https://unstop.com'+link.get('href'))
    return link_list    

In [15]:
links = scrape()
l2 = [0 for i in links]

# yet to extract the contact information till then creating a dummy array and giving the output in excel
df = pd.DataFrame({'hackathon links': links, 'Contact_information': l2})
df.to_excel('test.xlsx', sheet_name='sheet1', index=False)

## Trying to figure out how to extract contact information:

### Stuck here: [created a forum link](https://forum.webscraper.io/t/extract-contact-information-from-website-but-stuck/10543)

In [13]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get("https://unstop.com/hackathon/weave-the-web-powered-by-edot-solutions-techyon-2022-padre-conceicao-college-of-engineering-salcete-goa-462438")
stages = driver.find_elements_by_xpath("/html/body/div[1]/main/app-public-competition/div/div[2]/div[2]/div/div[1]/div[3]/div/app-dates-and-contacts/section/div/div[2]/ul")
print(stages)


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 96.0.4664
[WDM] - Get LATEST chromedriver version for 96.0.4664 google-chrome
[WDM] - Driver [/home/rohit/.wdm/drivers/chromedriver/linux64/96.0.4664.45/chromedriver] found in cache


[]


## For contacts extract all of the content under the class "contact ng-star-inserted"

In [ ]:
#tab-dates > div > app-dates-and-contacts > section > div > div:nth-child(2) > ul